In [1]:
import pandas as pd
import datetime
from os import listdir
import os, os.path
from os.path import isfile, join
import numpy as np
import csv
import re
import pandas as pd
import pm4py
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.log.exporter.xes import exporter as xes_exporter
from pm4py.objects.log.util import dataframe_utils
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def overlapIndex(list1,list2):
    set1=set(list1)
    set2=set(list2)
    intersection = set1.intersection(set2)
    len1=len(list(set1))
    len2=len(list(set2))
    overlapIndex=float(len(list(intersection)))/min([len1,len2])
    return overlapIndex

In [5]:
df_categorias_autores=pd.read_csv("logCT2.csv",index_col=0)
df_categorias_autores.columns

Index(['case:concept:name', 'concept:name', 'Specific operation',
       'time:timestamp', 'Report', 'Question', 'Year', 'Category', 'Type'],
      dtype='object')

In [6]:
df_categorias_autores["questYear"]=[rep+" "+str(year) for rep,year in zip(df_categorias_autores['Question'],df_categorias_autores['Year'])]

In [7]:
df_categorias_autores

,case:concept:name,concept:name,Specific operation,time:timestamp,Report,Question,Year,Category,Type,questYear
0,A1 P2 BPI 2020,Calculate cycle time,Calculate cycle time of the whole process for ...,2021-10-19 12:47:13.651481,A1,P2,2020,Academic,Analyze Cycle Time,P2 2020
1,A1 P2 BPI 2020,Compare cycle time,,2021-10-19 12:47:13.659459,A1,P2,2020,Academic,Analyze Cycle Time,P2 2020
2,A2 P2 BPI 2020,Calculate cycle time,Get cycle time from the previous question,2021-10-19 12:47:13.666495,A2,P2,2020,Student,Analyze Cycle Time,P2 2020
3,A2 P2 BPI 2020,Compare cycle time,,2021-10-19 12:47:13.666495,A2,P2,2020,Student,Analyze Cycle Time,P2 2020
4,A7 P2 BPI 2020,Calculate throughput,,2021-10-19 12:47:13.682150,A7,P2,2020,Academic,Analyze Cycle Time,P2 2020
...,...,...,...,...,...,...,...,...,...,...
943,A19 P1 BPI 2020,Group traces,Group traces by year,2021-10-19 12:47:20.534786,A19,P1,2020,Professional,Manipulation,P1 2020
944,A19 P1 BPI 2020,Group traces,Group traces by activities,2021-10-19 12:47:20.534786,A19,P1,2020,Professional,Manipulation,P1 2020
945,A19 P1 BPI 2020,Calculate statistics,Calculate Statistics of activities,2021-10-19 12:47:20.557196,A19,P1,2020,Professional,Calculate statistics,P1 2020
946,A19 P1 BPI 2020,Calculate throughput,,2021-10-19 12:47:20.567144,A19,P1,2020,Professional,Analyze Cycle Time,P1 2020


In [20]:
preguntasAgrupadas=df_categorias_autores.groupby("questYear").apply(lambda x: x['concept:name'].unique())

In [28]:
index=preguntasAgrupadas.index

In [53]:
valueIndexes=[]
comparisons=[]
question1=[]
question2=[]
for i in range(len(preguntasAgrupadas)-1):
    listaOpQY1=preguntasAgrupadas[i]
    qy1=index[i]

    for j in range(i+1,len(preguntasAgrupadas)):
        listaOpQY2=preguntasAgrupadas[j]
        qy2=index[j]
        overIndex=overlapIndex(listaOpQY1,listaOpQY2)
        valueIndexes.append(overIndex)
        comparisons.append(qy1+"-"+qy2)
        question1.append(qy1)
        question2.append(qy2)
        
dict_df={"parPreguntas":comparisons,"First question":question1,"Second question":question2,"Comparison index":valueIndexes,}
df_analisis_preguntas=pd.DataFrame.from_dict(dict_df)

In [ ]:
def comparePairsDF(df,func,qy):
    
    parAnalysis=[]
    categories=[]
    index=[]
    questionYear=[]
    
    idsAnalisis=list(set(df["repYear"]))
    cat1=""
    cat2=""
    
    for i in range(len(idsAnalisis)-1):
        print(idsAnalisis)
        print("----------------------")
        ident=idsAnalisis[i]
        operacionesIdent1=df[df["repYear"]==ident]['concept:name']
        #idsAnalisis.remove(ident)
        print(ident)
        print("----------------------")
        print("segunda lista")
        print(idsAnalisis)
        cat1=ident

        for j in range(i+1,len(idsAnalisis)-1):
            ident2=idsAnalisis[j]
            print("identificador dos")
            print(ident2)

            operacionesIdent2=df[df["repYear"]==ident2]['concept:name']

            cats=cat1+"-"+cat2

            categories.append(cats)
            index.append(func(operacionesIdent1,operacionesIdent2))
            parAnalysis.append(ident+"-"+ident2)
            questionYear.append(qy)
            
    dict_df={"parAnalisis":parAnalysis,"Categories":categories,"Comparison index":index,"questionYear":questionYear}
    df_analisis=pd.DataFrame.from_dict(dict_df)
    #print(df_analisis)
    return df_analisis

In [ ]:
agrupaciones_años_preguntas=df_categorias_autores.groupby(["Question","Year"])
dict_df_analysis={"parAnalisis":[],"Categories":[],"Comparison index":[],"questionYear":[]}
df_analisis_similarity=pd.DataFrame.from_dict(dict_df_analysis)

for name, df_group in agrupaciones_años_preguntas:
    #print(df_group)
    df_año_pregunta=comparePairsDF(df_group,overlapIndex,name)
    df_analisis_similarity=pd.concat([df_analisis_similarity,df_año_pregunta])